In [1]:
import numpy as np

import pandas as pd

#import altair as alt

import bokeh
import bokeh.plotting
from bokeh.plotting import ColumnDataSource
from bokeh.models import LabelSet
import bokeh.io
bokeh.io.output_notebook()

Loading BokehJS ...

In [2]:
df = pd.read_excel('190829_Deuterium_Transfer_Peak_Areas.xlsx')

In [3]:
df.tail()

,Species,WellNumber,Peak ID,Ret Time,Start Tm,End Tm,m/z,Area,Area Percent,Height,Height Percent,A/H,Type
61,L,20,D50C24,9.503,9.490,9.520,TIC,348,0.00,489,0.01,0.71,U
62,L,20,D62C30,12.330,12.325,12.420,TIC,187498,2.34,82462,1.06,2.27,U
63,P,20,C18,7.439,7.410,7.465,TIC,7725224,98.13,7500449,99.14,1.03,U
64,P,20,D50C24,9.470,9.465,9.515,TIC,221,0.00,70,0.00,3.16,U
65,P,20,D62C30,12.397,12.325,12.420,TIC,147180,1.87,64997,0.86,2.26,U


In [4]:
contact_time=[0.3613203502970176,
              0.5187090452569049,
              0.6402841570212505,
              0.4064792700104109,
              0.12009308592075449,
              0.29994488333639535,
              0.9203502970175761,
              0.2672300814501807,
              0.2902321023945128,
              0.3128544307673464]

well_nums=[1,2,3,6,7,13,14,16,17,20]

df_contact_time=pd.DataFrame({'Contact Time':contact_time,'WellNumber':well_nums})

In [5]:
c18_Areas = np.repeat(df['Area'].loc[df['Peak ID'] == 'C18'],3).reset_index().drop("index", axis=1)
df['Normalized Area'] = np.divide(df['Area'],c18_Areas['Area'])
df['Hydrocarbon amount'] = df['Normalized Area']*25
df=pd.merge(df,df_contact_time)
df['Time Normalized CHC Amount'] = df['Hydrocarbon amount']/df['Contact Time']
df.tail()

,Species,WellNumber,Peak ID,Ret Time,Start Tm,End Tm,m/z,Area,Area Percent,Height,Height Percent,A/H,Type,Normalized Area,Hydrocarbon amount,Contact Time,Time Normalized CHC Amount
55,L,20,D50C24,9.503,9.490,9.520,TIC,348,0.00,489,0.01,0.71,U,0.000044,0.001110,0.312854,0.003549
56,L,20,D62C30,12.330,12.325,12.420,TIC,187498,2.34,82462,1.06,2.27,U,0.023932,0.598295,0.312854,1.912376
57,P,20,C18,7.439,7.410,7.465,TIC,7725224,98.13,7500449,99.14,1.03,U,1.000000,25.000000,0.312854,79.909369
58,P,20,D50C24,9.470,9.465,9.515,TIC,221,0.00,70,0.00,3.16,U,0.000029,0.000715,0.312854,0.002286
59,P,20,D62C30,12.397,12.325,12.420,TIC,147180,1.87,64997,0.86,2.26,U,0.019052,0.476297,0.312854,1.522423


In [6]:
df_C24 = df.loc[df['Peak ID'] == 'D50C24']

p = bokeh.plotting.figure(plot_width=600,
                          plot_height=600,
                          title='C24',
                          x_axis_label='Species',
                          y_axis_label='ng D50C24',
                          y_axis_type='log')


for i in range(20):
    curSample = df_C24.loc[df_C24['WellNumber'] == i+1]
    if (any((curSample['Species'] == 'S') & (curSample['Type'] == 'U'))):
        curLiometopum=curSample['Hydrocarbon amount'].loc[curSample['Species']=='L']
        curSceptobius=curSample['Hydrocarbon amount'].loc[curSample['Species']=='S']
        
        p.circle(1,curLiometopum,color='Blue',legend='Liometopum',line_width=4)
        p.circle(2,curSceptobius,color='Red',legend='Sceptobius',line_width=4)
        p.line([1,2],[curLiometopum,curSceptobius],color='black')
        
        source = ColumnDataSource(
            pd.DataFrame(
                dict(
                    normalized_area=[curLiometopum.values],
                    position=[1],
                    labels=[str(i+1)]
                )
            )
        )
        
        label=bokeh.models.LabelSet(x='position',y='normalized_area',text='labels',level='glyph',
                                   x_offset=-15, y_offset=-10, source=source, render_mode='canvas')
        p.add_layout(label)
                    
    if (any((curSample['Species'] == 'S') & (curSample['Type'] == 'L'))):
        curLiometopum=curSample['Hydrocarbon amount'].loc[curSample['Species']=='L']
        curSceptobius=curSample['Hydrocarbon amount'].loc[curSample['Species']=='S']
        
        p.circle(3,curLiometopum,color='Blue',legend='Liometopum',line_width=4)
        p.circle(4,curSceptobius,color='Red',legend='Sceptobius',line_width=4)
        p.line([3,4],[curLiometopum,curSceptobius],color='black')
        
        source = ColumnDataSource(
            pd.DataFrame(
                dict(
                    normalized_area=[curLiometopum.values],
                    position=[3],
                    labels=[str(i+1)]
                )
            )
        )
        
        label=bokeh.models.LabelSet(x='position',y='normalized_area',text='labels',level='glyph',
                                   x_offset=-15, y_offset=-10, source=source, render_mode='canvas')
        p.add_layout(label)
        
    if (any((curSample['Species'] == 'P') & (curSample['Type'] == 'U'))):
        curLiometopum=curSample['Hydrocarbon amount'].loc[curSample['Species']=='L']
        curPlatyusa=curSample['Hydrocarbon amount'].loc[curSample['Species']=='P']
        
        p.circle(5,curLiometopum,color='Blue',legend='Liometopum',line_width=4)
        p.circle(6,curPlatyusa,color='Green',legend='Platyusa',line_width=4)
        p.line([5,6],[curLiometopum,curPlatyusa],color='black')
        
        source = ColumnDataSource(
            pd.DataFrame(
                dict(
                    normalized_area=[curLiometopum.values],
                    position=[5],
                    labels=[str(i+1)]
                )
            )
        )
        
        label=bokeh.models.LabelSet(x='position',y='normalized_area',text='labels',level='glyph',
                                   x_offset=-20, y_offset=-10, source=source, render_mode='canvas')
        p.add_layout(label)
        
    if (any((curSample['Species'] == 'P') & (curSample['Type'] == 'L'))):
        curLiometopum=curSample['Hydrocarbon amount'].loc[curSample['Species']=='L']
        curPlatyusa=curSample['Hydrocarbon amount'].loc[curSample['Species']=='P']
        
        p.circle(7,curLiometopum,color='Blue',legend='Liometopum',line_width=4)
        p.circle(8,curPlatyusa,color='Green',legend='Platyusa',line_width=4)
        p.line([7,8],[curLiometopum,curPlatyusa],color='black')
        
        source = ColumnDataSource(
            pd.DataFrame(
                dict(
                    normalized_area=[curLiometopum.values],
                    position=[7],
                    labels=[str(i+1)]
                )
            )
        )
        
        label=bokeh.models.LabelSet(x='position',y='normalized_area',text='labels',level='glyph',
                                   x_offset=-20, y_offset=-10, source=source, render_mode='canvas')
        p.add_layout(label)

p.legend.location = "top_left"


p.xaxis.visible = False
p.xgrid.visible = False
bokeh.io.show(p)

In [7]:
df_C30 = df.loc[df['Peak ID'] == 'D62C30']

p = bokeh.plotting.figure(plot_width=600,
                          plot_height=600,
                          title='C30',
                          x_axis_label='Species',
                          y_axis_label='ng D62C30',
                          y_axis_type='log')


for i in range(len(df_C30)):
    curSample = df_C30.loc[df_C30['WellNumber'] == i+1]
    if (any((curSample['Species'] == 'S') & (curSample['Type'] == 'U'))):
        curLiometopum=curSample['Hydrocarbon amount'].loc[curSample['Species']=='L']
        curSceptobius=curSample['Hydrocarbon amount'].loc[curSample['Species']=='S']
        
        p.circle(1,curLiometopum,color='Blue',legend='Liometopum',line_width=4)
        p.circle(2,curSceptobius,color='Red',legend='Sceptobius',line_width=4)
        p.line([1,2],[curLiometopum,curSceptobius],color='black')
        
        source = ColumnDataSource(
            pd.DataFrame(
                dict(
                    normalized_area=[curLiometopum.values],
                   
                    position=[1],
                    labels=[str(i+1)]
                )
            )
        )
        
        label=bokeh.models.LabelSet(x='position',y='normalized_area',text='labels',level='glyph',
                                   x_offset=-15, y_offset=-10, source=source, render_mode='canvas')
        p.add_layout(label)
                    
    if (any((curSample['Species'] == 'S') & (curSample['Type'] == 'L'))):
        curLiometopum=curSample['Hydrocarbon amount'].loc[curSample['Species']=='L']
        curSceptobius=curSample['Hydrocarbon amount'].loc[curSample['Species']=='S']
        
        p.circle(3,curLiometopum,color='Blue',legend='Liometopum',line_width=4)
        p.circle(4,curSceptobius,color='Red',legend='Sceptobius',line_width=4)
        p.line([3,4],[curLiometopum,curSceptobius],color='black')
        
        source = ColumnDataSource(
            pd.DataFrame(
                dict(
                    normalized_area=[curLiometopum.values],
                    position=[3],
                    labels=[str(i+1)]
                )
            )
        )
        
        label=bokeh.models.LabelSet(x='position',y='normalized_area',text='labels',level='glyph',
                                   x_offset=-15, y_offset=-10, source=source, render_mode='canvas')
        p.add_layout(label)
        
    if (any((curSample['Species'] == 'P') & (curSample['Type'] == 'U'))):
        curLiometopum=curSample['Hydrocarbon amount'].loc[curSample['Species']=='L']
        curPlatyusa=curSample['Hydrocarbon amount'].loc[curSample['Species']=='P']
        
        p.circle(5,curLiometopum,color='Blue',legend='Liometopum',line_width=4)
        p.circle(6,curPlatyusa,color='Green',legend='Platyusa',line_width=4)
        p.line([5,6],[curLiometopum,curPlatyusa],color='black')
        
        source = ColumnDataSource(
            pd.DataFrame(
                dict(
                    normalized_area=[curLiometopum.values],
                    position=[5],
                    labels=[str(i+1)]
                )
            )
        )
        
        label=bokeh.models.LabelSet(x='position',y='normalized_area',text='labels',level='glyph',
                                   x_offset=-20, y_offset=-10, source=source, render_mode='canvas')
        p.add_layout(label)
        
    if (any((curSample['Species'] == 'P') & (curSample['Type'] == 'L'))):
        curLiometopum=curSample['Hydrocarbon amount'].loc[curSample['Species']=='L']
        curPlatyusa=curSample['Hydrocarbon amount'].loc[curSample['Species']=='P']
        
        p.circle(7,curLiometopum,color='Blue',legend='Liometopum',line_width=4)
        p.circle(8,curPlatyusa,color='Green',legend='Platyusa',line_width=4)
        p.line([7,8],[curLiometopum,curPlatyusa],color='black')
        
        source = ColumnDataSource(
            pd.DataFrame(
                dict(
                    normalized_area=[curLiometopum.values],
                    position=[7],
                    labels=[str(i+1)]
                )
            )
        )
        
        label=bokeh.models.LabelSet(x='position',y='normalized_area',text='labels',level='glyph',
                                   x_offset=-20, y_offset=-10, source=source, render_mode='canvas')
        p.add_layout(label)
        
p.legend.location = "top_left"
p.xaxis.visible = False
p.xgrid.visible = False
bokeh.io.show(p)

In [8]:
df_C24 = df.loc[df['Peak ID'] == 'D50C24']

p = bokeh.plotting.figure(plot_width=600,
                          plot_height=600,
                          title='C24',
                          x_axis_label='Species',
                          y_axis_label='ng D50C24',
                          y_axis_type='log')


for i in range(20):
    curSample = df_C24.loc[df_C24['WellNumber'] == i+1]
    if (any((curSample['Species'] == 'S') & (curSample['Type'] == 'U'))):
        curLiometopum=curSample['Hydrocarbon amount'].loc[curSample['Species']=='L']
        curSceptobius=curSample['Time Normalized CHC Amount'].loc[curSample['Species']=='S']
        
        p.circle(1,curLiometopum,color='Blue',legend='Liometopum',line_width=4)
        p.circle(2,curSceptobius,color='Red',legend='Sceptobius',line_width=4)
        p.line([1,2],[curLiometopum,curSceptobius],color='black')
        
        source = ColumnDataSource(
            pd.DataFrame(
                dict(
                    normalized_area=[curLiometopum.values],
                    position=[1],
                    labels=[str(i+1)]
                )
            )
        )
        
        label=bokeh.models.LabelSet(x='position',y='normalized_area',text='labels',level='glyph',
                                   x_offset=-15, y_offset=-10, source=source, render_mode='canvas')
        p.add_layout(label)
                    
    if (any((curSample['Species'] == 'S') & (curSample['Type'] == 'L'))):
        curLiometopum=curSample['Hydrocarbon amount'].loc[curSample['Species']=='L']
        curSceptobius=curSample['Time Normalized CHC Amount'].loc[curSample['Species']=='S']
        
        p.circle(3,curLiometopum,color='Blue',legend='Liometopum',line_width=4)
        p.circle(4,curSceptobius,color='Red',legend='Sceptobius',line_width=4)
        p.line([3,4],[curLiometopum,curSceptobius],color='black')
        
        source = ColumnDataSource(
            pd.DataFrame(
                dict(
                    normalized_area=[curLiometopum.values],
                    position=[3],
                    labels=[str(i+1)]
                )
            )
        )
        
        label=bokeh.models.LabelSet(x='position',y='normalized_area',text='labels',level='glyph',
                                   x_offset=-15, y_offset=-10, source=source, render_mode='canvas')
        p.add_layout(label)
        
    if (any((curSample['Species'] == 'P') & (curSample['Type'] == 'U'))):
        curLiometopum=curSample['Hydrocarbon amount'].loc[curSample['Species']=='L']
        curPlatyusa=curSample['Time Normalized CHC Amount'].loc[curSample['Species']=='P']
        
        p.circle(5,curLiometopum,color='Blue',legend='Liometopum',line_width=4)
        p.circle(6,curPlatyusa,color='Green',legend='Platyusa',line_width=4)
        p.line([5,6],[curLiometopum,curPlatyusa],color='black')
        
        source = ColumnDataSource(
            pd.DataFrame(
                dict(
                    normalized_area=[curLiometopum.values],
                    position=[5],
                    labels=[str(i+1)]
                )
            )
        )
        
        label=bokeh.models.LabelSet(x='position',y='normalized_area',text='labels',level='glyph',
                                   x_offset=-15, y_offset=-10, source=source, render_mode='canvas')
        p.add_layout(label)
                    
    if (any((curSample['Species'] == 'P') & (curSample['Type'] == 'L'))):
        curLiometopum=curSample['Hydrocarbon amount'].loc[curSample['Species']=='L']
        curPlatyusa=curSample['Time Normalized CHC Amount'].loc[curSample['Species']=='P']
        
        p.circle(7,curLiometopum,color='Blue',legend='Liometopum',line_width=4)
        p.circle(8,curPlatyusa,color='Green',legend='Platyusa',line_width=4)
        p.line([7,8],[curLiometopum,curPlatyusa],color='black')
        
        source = ColumnDataSource(
            pd.DataFrame(
                dict(
                    normalized_area=[curLiometopum.values],
                    position=[7],
                    labels=[str(i+1)]
                )
            )
        )
        
        label=bokeh.models.LabelSet(x='position',y='normalized_area',text='labels',level='glyph',
                                   x_offset=-15, y_offset=-10, source=source, render_mode='canvas')
        p.add_layout(label)

p.legend.location = "top_left"


p.xaxis.visible = False
p.xgrid.visible = False
bokeh.io.show(p)

In [9]:
df_C30 = df.loc[df['Peak ID'] == 'D62C30']

p = bokeh.plotting.figure(plot_width=600,
                          plot_height=600,
                          title='C30',
                          x_axis_label='Species',
                          y_axis_label='ng D62C30',
                          y_axis_type='log')


for i in range(20):
    curSample = df_C30.loc[df_C30['WellNumber'] == i+1]
    if (any((curSample['Species'] == 'S') & (curSample['Type'] == 'U'))):
        curLiometopum=curSample['Hydrocarbon amount'].loc[curSample['Species']=='L']
        curSceptobius=curSample['Time Normalized CHC Amount'].loc[curSample['Species']=='S']
        
        p.circle(1,curLiometopum,color='Blue',legend='Liometopum',line_width=4)
        p.circle(2,curSceptobius,color='Red',legend='Sceptobius',line_width=4)
        p.line([1,2],[curLiometopum,curSceptobius],color='black')
        
        source = ColumnDataSource(
            pd.DataFrame(
                dict(
                    normalized_area=[curLiometopum.values],
                   
                    position=[1],
                    labels=[str(i+1)]
                )
            )
        )
        
        label=bokeh.models.LabelSet(x='position',y='normalized_area',text='labels',level='glyph',
                                   x_offset=-15, y_offset=-10, source=source, render_mode='canvas')
        p.add_layout(label)
                    
    if (any((curSample['Species'] == 'S') & (curSample['Type'] == 'L'))):
        curLiometopum=curSample['Hydrocarbon amount'].loc[curSample['Species']=='L']
        curSceptobius=curSample['Time Normalized CHC Amount'].loc[curSample['Species']=='S']
        
        p.circle(3,curLiometopum,color='Blue',legend='Liometopum',line_width=4)
        p.circle(4,curSceptobius,color='Red',legend='Sceptobius',line_width=4)
        p.line([3,4],[curLiometopum,curSceptobius],color='black')
        
        source = ColumnDataSource(
            pd.DataFrame(
                dict(
                    normalized_area=[curLiometopum.values],
                    position=[3],
                    labels=[str(i+1)]
                )
            )
        )
        
        label=bokeh.models.LabelSet(x='position',y='normalized_area',text='labels',level='glyph',
                                   x_offset=-15, y_offset=-10, source=source, render_mode='canvas')
        p.add_layout(label)
        
        
    if (any((curSample['Species'] == 'P') & (curSample['Type'] == 'U'))):
        curLiometopum=curSample['Hydrocarbon amount'].loc[curSample['Species']=='L']
        curPlatyusa=curSample['Time Normalized CHC Amount'].loc[curSample['Species']=='P']
        
        p.circle(5,curLiometopum,color='Blue',legend='Liometopum',line_width=4)
        p.circle(6,curPlatyusa,color='Green',legend='Platyusa',line_width=4)
        p.line([5,6],[curLiometopum,curPlatyusa],color='black')
        
        source = ColumnDataSource(
            pd.DataFrame(
                dict(
                    normalized_area=[curLiometopum.values],
                    position=[5],
                    labels=[str(i+1)]
                )
            )
        )
        
        label=bokeh.models.LabelSet(x='position',y='normalized_area',text='labels',level='glyph',
                                   x_offset=-15, y_offset=-10, source=source, render_mode='canvas')
        p.add_layout(label)
                    
    if (any((curSample['Species'] == 'P') & (curSample['Type'] == 'L'))):
        curLiometopum=curSample['Hydrocarbon amount'].loc[curSample['Species']=='L']
        curPlatyusa=curSample['Time Normalized CHC Amount'].loc[curSample['Species']=='P']
        
        p.circle(7,curLiometopum,color='Blue',legend='Liometopum',line_width=4)
        p.circle(8,curPlatyusa,color='Green',legend='Platyusa',line_width=4)
        p.line([7,8],[curLiometopum,curPlatyusa],color='black')
        
        source = ColumnDataSource(
            pd.DataFrame(
                dict(
                    normalized_area=[curLiometopum.values],
                    position=[7],
                    labels=[str(i+1)]
                )
            )
        )
        
        label=bokeh.models.LabelSet(x='position',y='normalized_area',text='labels',level='glyph',
                                   x_offset=-15, y_offset=-10, source=source, render_mode='canvas')
        p.add_layout(label)

p.legend.location = "top_left"
p.xaxis.visible = False
p.xgrid.visible = False
bokeh.io.show(p)

In [10]:
df_C30

,Species,WellNumber,Peak ID,Ret Time,Start Tm,End Tm,m/z,Area,Area Percent,Height,Height Percent,A/H,Type,Normalized Area,Hydrocarbon amount,Contact Time,Time Normalized CHC Amount
2,L,1,D62C30,12.368,12.325,12.415,TIC,729479,7.10,289316,3.16,2.52,L,0.094365,2.359137,0.361320,6.529210
5,S,1,D62C30,12.368,12.325,12.415,TIC,27753,0.37,10256,0.14,2.71,L,0.003760,0.094008,0.361320,0.260180
8,L,2,D62C30,12.368,12.325,12.415,TIC,360812,3.93,138704,1.72,2.60,L,0.045804,1.145108,0.518709,2.207611
11,S,2,D62C30,12.335,12.325,12.415,TIC,47795,0.62,14160,0.20,3.38,L,0.006326,0.158147,0.518709,0.304886
14,L,3,D62C30,12.367,12.325,12.415,TIC,608962,6.29,236229,2.69,2.58,L,0.076949,1.923731,0.640284,3.004495
17,S,3,D62C30,12.366,12.325,12.415,TIC,42459,0.53,15935,0.22,2.66,L,0.005390,0.134745,0.640284,0.210446
20,L,6,D62C30,12.333,12.325,12.415,TIC,91806,1.13,32846,0.43,2.80,U,0.011454,0.286348,0.406479,0.704460
23,S,6,D62C30,12.333,12.325,12.415,TIC,1630,0.02,1283,0.02,1.27,U,0.000213,0.005316,0.406479,0.013078
26,L,7,D62C30,12.333,12.325,12.415,TIC,38040,0.51,14186,0.19,2.68,U,0.005167,0.129180,0.120093,1.075667
29,S,7,D62C30,12.331,12.325,12.415,TIC,4368,0.06,1913,0.03,2.28,U,0.000596,0.014912,0.120093,0.124171
